# Onebox helper

**User profile:**
- start search center
- preferred user language
- defaults: local ip related

**Results limits:**
```
OneBoxMap.default_terms_limit = 3
OneBoxMap.default_results_limit = 10
OneBoxMap.default_suggestions_limit = 5
```

In [ ]:
from search.widgets import OneBoxMap

OneBoxMap().run()

## Various designs

`Design.two` and `SearchResultJson`

In [ ]:
from search.widgets import Design, SearchResultJson

app = OneBoxMap(
    language='en', 
    latitude=40.71455, longitude=-74.00714, 
    design=Design.one, 
    resultlist_class=SearchResultJson)
app.run()

## Adapt query parser

`OneBoxExt` and category near category

In [ ]:
from search.widgets import OneBoxExt

OneBoxExt().run()

In [ ]:
from search.widgets import OneBoxMap
from IPython.display import display as Idisplay
from ipywidgets import Output
from aiohttp import ClientSession
import nest_asyncio

from search.core import OneBoxBase
from search.widgets.query import SubmittableTextBox, TermsButtons, QuerySimpleParser
from search.widgets.response import SearchFeatureCollection, SearchResultButtons, PositionMap, SearchResultJson, SearchResultRadioButtons
from search.widgets.design import Design

from typing import Callable, ClassVar, Tuple
import asyncio

class OneBoxExt(OneBoxMap):
    default_autosuggest_query_params = {'show': 'ontologyDetails,expandedOntologies'}

    def __init__(self,
                 language: str=None,
                 latitude: float=None, longitude: float=None,
                 api_key: str=None,
                 results_limit: int=None,
                 suggestions_limit: int=None,
                 terms_limit: int=None,
                 design: Callable=None,
                 resultlist_class: ClassVar=None,
                 autosuggest_automatic_recenter: bool=False,
                 **kwargs):

        super().__init__(language, latitude, longitude, api_key,
                         results_limit, suggestions_limit, terms_limit,
                         design, resultlist_class, autosuggest_automatic_recenter, **kwargs)
        self.find_ontology = None
        self.find_query = None
        self.near_ontology = None
        self.near_query = None

    async def handle_key_strokes(self):
        """
        This method is called for each key stroke in the one box search Text form.
        """
        params = {"show": 'details,expandedOntologies', "types": "categoryQuery,chainQuery,place,city,country"}
        out = Output()
        async with ClientSession(raise_for_status=True, headers=OneBoxMap.default_headers) as session:
            while True:
                q = await self.wait_for_new_key_stroke()
                if q is None:
                    break
                if q:

                    latitude, longitude = self.get_search_center()
                    autosuggest_resp = None
                    conjunction, conjunction_mode, query_with_conjunction_head, query_with_conjunction_tail = QuerySimpleParser(q).get_conjunction_mode(self.language)
                    print(conjunction_mode)
                    if conjunction_mode == QuerySimpleParser.ConjunctionMode.incomplete_conjunction:
                        #self.find_query = query_with_conjunction_head
                        find_resp = await asyncio.ensure_future(self.autosuggest(session, query_with_conjunction_head, latitude, longitude, **self.autosuggest_query_params))
                        if len(find_resp["items"]) > 0 and find_resp["items"][0]["resultType"] == "categoryQuery" and "relationship" not in find_resp["items"][0]:
                            self.find_query = query_with_conjunction_head
                            self.find_ontology = autosuggest_resp["items"][0]
                            print(self.find_ontology["title"])
                        else:
                            self.find_query = None
                            self.find_ontology = None
                        self.near_query = None
                        self.near_ontology = None
                        autosuggest_resp["queryTerms"] = ([{"term": conjunction}] + autosuggest_resp["queryTerms"])[:OneBoxMap.default_terms_limit]
                    elif conjunction_mode == QuerySimpleParser.ConjunctionMode.conjunction_surrounded:
                        if self.find_query is None:
                            find_resp = await asyncio.ensure_future(self.autosuggest(session, query_with_conjunction_head, latitude, longitude, **params))
                            if len(find_resp["items"]) > 0 and find_resp["items"][0]["resultType"] == "categoryQuery" and "relationship" not in find_resp["items"][0]:
                                self.find_query = query_with_conjunction_head
                                self.find_ontology = find_resp["items"][0]
                                print(self.find_ontology["title"])
                            else:
                                self.find_query = None
                                self.find_ontology = None

                        if self.find_ontology:
                            near_resp = await asyncio.ensure_future(self.autosuggest(session, query_with_conjunction_tail, latitude, longitude, **params))
                            if len(near_resp["items"]) > 0 and near_resp["items"][0]["resultType"] == "categoryQuery" and "relationship" not in near_resp["items"][0]:
                                self.near_query = query_with_conjunction_tail
                                self.near_ontology = near_resp["items"][0]
                                # Replace the query below with the calls to Location Graph
                                autosuggest_resp = await asyncio.ensure_future(self.autosuggest(session, q, latitude, longitude, **self.autosuggest_query_params))
                                autosuggest_resp["items"] = ([{"title": f"{self.find_ontology['title']} {conjunction} {near_resp['items'][0]['title']}",
                                                               "id": f"{self.find_ontology['id']}:near:{self.near_ontology['id']}",
                                                               "resultType": "categoryQuery",
                                                               "href": f"{self.__class__.ds_url}?at={latitude},{longitude}&lang={self.language}&q={q}",
                                                               "highlights": {}}] + autosuggest_resp["items"])[:OneBoxMap.default_results_limit]
                            else:
                                autosuggest_resp = await asyncio.ensure_future(self.autosuggest(session, q, latitude, longitude, **self.autosuggest_query_params))
                            if near_resp["queryTerms"]:
                                autosuggest_resp["queryTerms"] = ([{"term": near_resp["queryTerms"][0]["term"]}] + autosuggest_resp["queryTerms"])[:OneBoxMap.default_terms_limit]
                    elif conjunction_mode != QuerySimpleParser.ConjunctionMode.conjunction_spaces_last and q.endswith(' ') and self.find_ontology:
                        autosuggest_resp = await asyncio.ensure_future(self.autosuggest(session, q, latitude, longitude, **self.autosuggest_query_params))
                        if len(autosuggest_resp["items"]) > 0 and autosuggest_resp["items"][0]["resultType"] == "categoryQuery" and "relationship" not in autosuggest_resp["items"][0]:
                            self.find_query = q
                            self.find_ontology = autosuggest_resp["items"][0]
                            print(self.find_ontology["title"])
                        else:
                            self.find_query = None
                            self.find_ontology = None
                        self.near_query = None
                        self.near_ontology = None
                        autosuggest_resp["queryTerms"] = [{"term": conjunction}]
                        autosuggest_resp = await asyncio.ensure_future(self.autosuggest(session, q, latitude, longitude, **self.autosuggest_query_params))
                        if len(autosuggest_resp["items"]) > 0 and autosuggest_resp["items"][0]["resultType"] == "categoryQuery" and "relationship" not in autosuggest_resp["items"][0]:
                            self.find_query = q
                            self.find_ontology = autosuggest_resp["items"][0]
                            print(self.find_ontology["title"])
                        else:
                            self.find_query = q
                            self.find_ontology = None
                        self.near_query = None
                        self.near_ontology = None

                    if autosuggest_resp is None:
                        autosuggest_resp = await asyncio.ensure_future(self.autosuggest(session, q, latitude, longitude, **self.autosuggest_query_params))
                        if len(autosuggest_resp["items"]) > 0 and autosuggest_resp["items"][0]["resultType"] == "categoryQuery" and "relationship" not in autosuggest_resp["items"][0]:
                            self.find_query = q
                            self.find_ontology = autosuggest_resp["items"][0]
                            print(self.find_ontology["title"])
                        else:
                            self.find_query = q
                            self.find_ontology = None
                            
                    self.handle_suggestion_list(autosuggest_resp)
                    if self.text.value.strip().endswith(conjunction):
                        self.query_terms_w.set([])

                else:
                    self.handle_empty_text_submission()

    def run(self):
        OneBoxMap.run(self, handle_key_strokes=self.handle_key_strokes)
        
OneBoxExt().run()